# 17 - POO: clases y objetos para modelar estado y comportamiento

## Objetivos de aprendizaje

En esta sesion aprenderas a:

1. Entender que problema resuelve la Programacion Orientada a Objetos (POO).
2. Distinguir con precision entre clase, objeto, atributo y metodo.
3. Construir clases con `__init__`, estado interno y comportamiento coherente.
4. Diferenciar atributos de instancia y atributos de clase.
5. Usar `@classmethod` y `@staticmethod` con criterio de diseno.
6. Mantener invariantes para evitar estados invalidos.
7. Aplicar encapsulacion practica con convenciones y `@property`.
8. Modelar colaboraciones entre objetos mediante composicion.
9. Detectar errores comunes de modelado antes de que lleguen a produccion.
10. Tomar decisiones de arquitectura: cuando usar clases y cuando no.


## 1. Problema que resuelve la POO

Hasta ahora has resuelto muchos problemas con funciones y estructuras basicas (`list`, `dict`, `tuple`).
Ese enfoque funciona bien, pero en sistemas medianos aparecen tres fricciones:

- El estado queda disperso en muchas variables sueltas.
- Las reglas de negocio se repiten en varios lugares.
- Es facil crear estados invalidos porque nadie "protege" los datos.

La POO propone agrupar **datos + comportamiento** en una sola unidad: el objeto.
Una clase define la plantilla; cada objeto es una instancia con su propio estado.


In [ ]:
# Enfoque procedural: estado y reglas repartidas.

productos = []

def registrar_producto(nombre, precio, stock):
    if precio < 0:
        raise ValueError("El precio no puede ser negativo")
    if stock < 0:
        raise ValueError("El stock no puede ser negativo")

    productos.append({"nombre": nombre, "precio": precio, "stock": stock})

def vender(nombre, cantidad):
    for p in productos:
        if p["nombre"] == nombre:
            if cantidad <= 0:
                raise ValueError("La cantidad debe ser positiva")
            if p["stock"] < cantidad:
                raise ValueError("Stock insuficiente")
            p["stock"] -= cantidad
            return
    raise ValueError("Producto no encontrado")

registrar_producto("teclado", 500, 10)
vender("teclado", 2)
print(productos)


## 2. Primera clase: una plantilla con identidad propia

Una clase te permite declarar, en un solo lugar:

- que datos tiene cada objeto,
- que operaciones puede ejecutar,
- y que reglas se deben respetar.

Piensa asi:

- **Clase**: plano de construccion.
- **Objeto**: construccion concreta hecha con ese plano.


In [ ]:
class Producto:
    def __init__(self, nombre, precio, stock):
        self.nombre = nombre
        self.precio = precio
        self.stock = stock

    def vender(self, cantidad):
        if cantidad <= 0:
            raise ValueError("La cantidad debe ser positiva")
        if self.stock < cantidad:
            raise ValueError("Stock insuficiente")
        self.stock -= cantidad

teclado = Producto("teclado", 500, 10)
mouse = Producto("mouse", 250, 20)

teclado.vender(3)
print(teclado.stock)  # 7
print(mouse.stock)    # 20


## 3. `self`: referencia al objeto actual

`self` no es una palabra reservada, pero por convencion siempre se usa ese nombre.
Representa "este objeto en particular".

Cuando escribes `teclado.vender(3)`, Python internamente llama:
`Producto.vender(teclado, 3)`.

Por eso cada objeto mantiene su propio estado.


In [ ]:
class ContadorClick:
    def __init__(self):
        self.total = 0

    def click(self):
        self.total += 1

boton_a = ContadorClick()
boton_b = ContadorClick()

boton_a.click()
boton_a.click()
boton_b.click()

print("A:", boton_a.total)  # 2
print("B:", boton_b.total)  # 1


## 4. `__init__`: constructor para estado inicial valido

`__init__` se ejecuta al crear una instancia.
No "crea" el objeto (eso lo hace `__new__`), pero si lo inicializa.

Buenas practicas en `__init__`:

1. Validar datos de entrada.
2. Dejar el objeto en estado consistente.
3. Evitar logica pesada o I/O innecesario.


In [ ]:
class Usuario:
    def __init__(self, nombre, email):
        nombre = nombre.strip()
        email = email.strip().lower()

        if not nombre:
            raise ValueError("Nombre obligatorio")
        if "@" not in email:
            raise ValueError("Email invalido")

        self.nombre = nombre
        self.email = email

ana = Usuario(" Ana ", "ANA@EJEMPLO.COM")
print(ana.nombre)  # Ana
print(ana.email)   # ana@ejemplo.com


## 5. Invariantes: reglas que siempre deben cumplirse

Una invariante es una condicion que debe ser verdadera en todo momento relevante del objeto.
Ejemplos:

- saldo >= 0
- porcentaje entre 0 y 100
- fecha_fin >= fecha_inicio

Si no modelas invariantes, un objeto puede existir en un estado absurdo y causar bugs en cadena.


In [ ]:
class TemperaturaCelsius:
    def __init__(self, grados):
        self._grados = None
        self.grados = grados  # Reutilizamos la validacion del setter

    @property
    def grados(self):
        return self._grados

    @grados.setter
    def grados(self, valor):
        if valor < -273.15:
            raise ValueError("No existe temperatura menor al cero absoluto")
        self._grados = float(valor)

ambiente = TemperaturaCelsius(22)
ambiente.grados = 19.5
print(ambiente.grados)


## 6. Atributos de instancia vs atributos de clase

- **Atributo de instancia**: vive en cada objeto (`self.x`).
- **Atributo de clase**: vive en la clase y se comparte entre instancias.

Error comun: usar atributo de clase cuando querias un dato propio de cada objeto.


In [ ]:
class CuentaServicio:
    iva = 0.16  # atributo de clase (compartido)

    def __init__(self, titular, consumo):
        self.titular = titular
        self.consumo = consumo  # atributo de instancia

    def total(self):
        return self.consumo * (1 + CuentaServicio.iva)

c1 = CuentaServicio("Luz", 100)
c2 = CuentaServicio("Agua", 250)

print(c1.total())
print(c2.total())

CuentaServicio.iva = 0.20
print(c1.total())  # cambia para todas las instancias


## 7. Metodos de instancia: comportamiento del objeto

Un metodo debe expresar una accion significativa del dominio.
Evita clases con puros getters/setters triviales sin comportamiento real.

Regla practica: si una operacion depende del estado interno, suele pertenecer al objeto.


In [ ]:
class Carrito:
    def __init__(self):
        self.items = []

    def agregar(self, nombre, precio, cantidad=1):
        if precio < 0:
            raise ValueError("Precio invalido")
        if cantidad <= 0:
            raise ValueError("Cantidad invalida")
        self.items.append({"nombre": nombre, "precio": precio, "cantidad": cantidad})

    def subtotal(self):
        return sum(item["precio"] * item["cantidad"] for item in self.items)

carrito = Carrito()
carrito.agregar("cuaderno", 35, 2)
carrito.agregar("lapiz", 12, 3)
print(carrito.subtotal())


## 8. `@classmethod` y `@staticmethod`

No todo metodo debe recibir `self`.

- `@classmethod`: recibe `cls` (la clase). Util para constructores alternos.
- `@staticmethod`: no recibe `self` ni `cls`. Es una funcion utilitaria relacionada semantica mente con la clase.

Usa estos decoradores por claridad conceptual, no solo por "variedad" sintactica.


In [ ]:
class Fecha:
    def __init__(self, anio, mes, dia):
        self.anio = anio
        self.mes = mes
        self.dia = dia

    @classmethod
    def desde_iso(cls, texto):
        anio, mes, dia = map(int, texto.split("-"))
        return cls(anio, mes, dia)

    @staticmethod
    def es_bisiesto(anio):
        return (anio % 4 == 0 and anio % 100 != 0) or (anio % 400 == 0)

f = Fecha.desde_iso("2026-02-17")
print(f.anio, f.mes, f.dia)
print(Fecha.es_bisiesto(2024))


## 9. Representacion del objeto: `__repr__`

Cuando imprimes una lista de objetos, una buena representacion facilita debugging.

- `__repr__` deberia ser clara y util para desarrolladores.
- Si no la defines, veras algo como `<__main__.Clase object at 0x...>`.


In [ ]:
class Tarea:
    def __init__(self, titulo, prioridad):
        self.titulo = titulo
        self.prioridad = prioridad

    def __repr__(self):
        return f"Tarea(titulo={self.titulo!r}, prioridad={self.prioridad})"

tareas = [Tarea("Estudiar POO", 1), Tarea("Hacer ejercicio", 2)]
print(tareas)


## 10. Encapsulacion practica: convenciones y `@property`

Python no impone encapsulacion estricta como otros lenguajes, pero ofrece herramientas utiles:

- `_atributo`: uso interno por convencion.
- `__atributo`: name mangling (evitar choques, no seguridad real).
- `@property`: controlar acceso sin romper interfaz de uso.

Objetivo: proteger invariantes sin volver la API incomoda.


In [ ]:
class CuentaBancaria:
    def __init__(self, titular, saldo_inicial=0):
        if saldo_inicial < 0:
            raise ValueError("Saldo inicial invalido")
        self.titular = titular
        self._saldo = float(saldo_inicial)

    @property
    def saldo(self):
        return self._saldo

    def depositar(self, monto):
        if monto <= 0:
            raise ValueError("Monto invalido")
        self._saldo += monto

    def retirar(self, monto):
        if monto <= 0:
            raise ValueError("Monto invalido")
        if monto > self._saldo:
            raise ValueError("Fondos insuficientes")
        self._saldo -= monto

cuenta = CuentaBancaria("Luis", 1000)
cuenta.depositar(250)
cuenta.retirar(100)
print(cuenta.saldo)


## 11. Identidad, mutabilidad y efectos colaterales

Dos variables pueden apuntar al mismo objeto.
Si el objeto es mutable, un cambio por una referencia se refleja en la otra.

Este punto es clave para evitar bugs "fantasma" al pasar objetos a funciones o metodos.


In [ ]:
class Caja:
    def __init__(self):
        self.valores = []

caja_original = Caja()
caja_alias = caja_original

caja_alias.valores.append("x")
print(caja_original.valores)  # ['x']
print(caja_original is caja_alias)  # True


## 12. Composicion: objetos que colaboran

Componer significa construir objetos grandes con objetos pequenos.
Generalmente es mas flexible que heredar "por costumbre".

Ejemplo: `Pedido` contiene varias `LineaPedido`, y cada linea referencia un `ProductoCatalogo`.


In [ ]:
class ProductoCatalogo:
    def __init__(self, sku, nombre, precio):
        self.sku = sku
        self.nombre = nombre
        self.precio = precio

class LineaPedido:
    def __init__(self, producto, cantidad):
        if cantidad <= 0:
            raise ValueError("Cantidad invalida")
        self.producto = producto
        self.cantidad = cantidad

    def subtotal(self):
        return self.producto.precio * self.cantidad

class Pedido:
    def __init__(self, cliente):
        self.cliente = cliente
        self.lineas = []

    def agregar_linea(self, producto, cantidad):
        self.lineas.append(LineaPedido(producto, cantidad))

    def total(self):
        return sum(linea.subtotal() for linea in self.lineas)

p1 = ProductoCatalogo("A1", "Monitor", 3200)
p2 = ProductoCatalogo("B2", "Cable HDMI", 180)

pedido = Pedido("Empresa XYZ")
pedido.agregar_linea(p1, 1)
pedido.agregar_linea(p2, 3)
print(pedido.total())


## 13. Caso real pequeno: inventario con reglas claras

La calidad de un modelo orientado a objetos no se mide por cuantas clases tiene,
sino por que tan bien expresa reglas del dominio.

En este ejemplo:
- un `ItemInventario` conoce su stock,
- `Inventario` coordina multiples items,
- y las reglas de entrada/salida se aplican en un punto central.


In [ ]:
class ItemInventario:
    def __init__(self, codigo, nombre, stock=0):
        if stock < 0:
            raise ValueError("Stock inicial invalido")
        self.codigo = codigo
        self.nombre = nombre
        self.stock = stock

    def entrar(self, cantidad):
        if cantidad <= 0:
            raise ValueError("Entrada invalida")
        self.stock += cantidad

    def salir(self, cantidad):
        if cantidad <= 0:
            raise ValueError("Salida invalida")
        if cantidad > self.stock:
            raise ValueError("Stock insuficiente")
        self.stock -= cantidad

class Inventario:
    def __init__(self):
        self._items = {}

    def agregar_item(self, item):
        if item.codigo in self._items:
            raise ValueError("Codigo duplicado")
        self._items[item.codigo] = item

    def item(self, codigo):
        return self._items[codigo]

inv = Inventario()
inv.agregar_item(ItemInventario("P-01", "Papel", 50))
inv.item("P-01").salir(5)
print(inv.item("P-01").stock)


## 14. Criterios de diseno: cuando conviene usar clases

Usa clases cuando:

1. Necesitas mantener estado entre operaciones.
2. Existen invariantes que deben protegerse.
3. El dominio tiene entidades con identidad y ciclo de vida.
4. Quieres encapsular reglas en una API coherente.

Prefiere funciones cuando:

1. La operacion es pura y stateless.
2. La logica es corta y no requiere identidad propia.
3. Una clase agregaria ceremonia sin claridad real.


## 15. Errores comunes en clases y objetos

1. Usar atributos de clase para datos que debian ser por instancia.
2. Exponer estado mutable interno sin control (`return self.lista`).
3. Meter demasiadas responsabilidades en una sola clase.
4. Crear clases como "contenedores de getters/setters" sin comportamiento.
5. Ignorar validaciones en `__init__` y permitir estados invalidos.
6. Modelar todo con herencia cuando composicion era suficiente.


## 16. Ejercicios de pensamiento cuidadoso

Estos ejercicios no buscan solo que "funcione" el codigo.
Buscan que argumentes decisiones de diseno, limites y tradeoffs.


### Ejercicio 1: analisis de dominio antes de programar

**Escenario**: Administras una plataforma de cursos en linea.
Necesitas representar estudiantes, cursos, inscripciones y calificaciones.

**Tarea**:
1. Propone al menos 4 clases con responsabilidades claras.
2. Define 2 invariantes por clase.
3. Explica una colaboracion entre objetos (quien llama a quien y por que).
4. Indica que parte resolverias con funciones en lugar de metodos y justifica.


In [ ]:
# Escribe aqui tu diseno inicial (puede ser texto en comentarios o codigo).
# Clase X: responsabilidad...
# Invariantes...


### Ejercicio 2: invariantes primero, implementacion despues

**Tarea**: Dise?a `CuentaAhorro` con estas reglas:

- saldo nunca negativo,
- limite diario de retiro,
- no permitir mas de N retiros al dia.

Antes de codificar, escribe:
1. estados validos,
2. estados invalidos,
3. estrategia de errores (que excepcion y en que casos).
Luego implementa.


In [ ]:
class CuentaAhorro:
    # Implementa aqui tu propuesta.
    pass


### Ejercicio 3: bug silencioso por atributo compartido

**Tarea**: detecta el bug, explica por que ocurre y corrigelo sin romper la interfaz publica.
Tambien propone una prueba minima que capture el problema.


In [ ]:
class Proyecto:
    tareas = []  # bug: se comparte entre todas las instancias

    def __init__(self, nombre):
        self.nombre = nombre

    def agregar_tarea(self, tarea):
        self.tareas.append(tarea)

p1 = Proyecto("Web")
p2 = Proyecto("Movil")
p1.agregar_tarea("Login")
print(p2.tareas)  # inesperado: tambien muestra ['Login']


### Ejercicio 4: dise?a API antes de escribir metodos

**Escenario**: `ReservaHotel`.

**Tarea**:
1. Define la interfaz publica minima (metodos y parametros).
2. Declara precondiciones y postcondiciones por metodo.
3. Implementa solo despues de escribir ese contrato.

Enfocate en claridad de API, no en cantidad de codigo.


In [ ]:
class ReservaHotel:
    # 1) Escribe contrato de metodos en comentarios.
    # 2) Implementa respetando invariantes.
    pass


### Ejercicio 5: separar responsabilidades

**Tarea**: La siguiente clase mezcla demasiadas cosas.
Identifica al menos 3 responsabilidades distintas y propone una refactorizacion en objetos colaboradores.


In [ ]:
class ReporteVentasTodoEnUno:
    def generar(self, ruta_csv):
        # Lee archivo
        # Limpia datos
        # Calcula metricas
        # Formatea texto
        # Envia correo
        # Guarda log
        pass

# Esboza aqui una propuesta de clases mas cohesivas.


### Ejercicio 6: modelar transiciones de estado

**Escenario**: Pedido con estados `nuevo`, `pagado`, `enviado`, `entregado`, `cancelado`.

**Tarea**:
1. Define transiciones validas e invalidas.
2. Implementa una clase que impida transiciones ilegales.
3. Explica que politica aplicarias para registrar historial de cambios.


In [ ]:
class PedidoEstado:
    # Implementa con una tabla de transiciones o con reglas explicitas.
    pass


### Ejercicio 7: efectos colaterales y encapsulacion

**Tarea**: Este diseno expone estado interno mutable.
1. Explica el riesgo.
2. Corrige el API para proteger invariantes.
3. Justifica si devuelves copia, tupla o vista de solo lectura.


In [ ]:
class Agenda:
    def __init__(self):
        self._eventos = []

    def eventos(self):
        return self._eventos  # Riesgo: quien recibe puede modificar internamente


### Ejercicio 8: pruebas como especificacion de negocio

**Tarea**: Implementa `CalculadoraDescuento` y escribe asserts que documenten reglas:

- descuento entre 0 y 100,
- total nunca negativo,
- redondeo definido (explica tu criterio).

La meta no es solo pasar pruebas, sino expresar decisiones del dominio.


In [ ]:
class CalculadoraDescuento:
    # Implementa aqui.
    pass

# Escribe asserts que documenten reglas y casos borde.


### Ejercicio 9: decision tecnica argumentada

Para cada caso, decide si usarias:
- funcion,
- clase,
- o combinacion de ambas.

Casos:
1. Validar formato de CURP.
2. Simular un semaforo con estado cambiante.
3. Convertir montos entre monedas con tasa fija.
4. Gestionar una sesion de usuario (inicio, expiracion, cierre).

Justifica en terminos de estado, invariantes y mantenibilidad.


In [ ]:
# Responde aqui con tu argumentacion tecnica.


### Ejercicio 10: mini diseno con tradeoffs explicitos

**Reto**: Dise?a un sistema pequeno de biblioteca con al menos estas clases:

- `Libro`
- `UsuarioBiblioteca`
- `Prestamo`

**Requisitos**:
1. No permitir dos prestamos activos del mismo libro.
2. Modelar vencimiento y posible multa.
3. Mantener una regla clara para "devolucion tardia".
4. Explicar una decision de diseno que dejaste fuera y por que.


In [ ]:
# Implementa una version minima funcional.
# Luego agrega una seccion de comentarios con tus tradeoffs.


## 17. Resumen de conceptos clave

1. Una clase modela una idea del dominio; un objeto es una instancia concreta.
2. `self` conecta comportamiento con estado.
3. `__init__` debe construir objetos validos desde el inicio.
4. Las invariantes evitan estados absurdos y bugs de cascada.
5. Atributos de clase y de instancia resuelven problemas distintos.
6. `@classmethod` y `@staticmethod` mejoran expresividad cuando se usan con criterio.
7. Encapsular no es ocultar todo: es proteger reglas sin sacrificar usabilidad.
8. Composicion suele dar disenos mas flexibles y mantenibles.
9. Un buen modelo orientado a objetos prioriza coherencia de dominio sobre cantidad de clases.


## 18. Reto final opcional

Disena un mini sistema de **gestion de gimnasio** con estas entidades:

1. `Membresia`
2. `Socio`
3. `ClaseGrupal`
4. `Inscripcion`

Incluye:

- reglas de cupo,
- vencimiento de membresia,
- politicas de cancelacion,
- y un breve analisis de que parte resolverias con funciones auxiliares.

Objetivo: practicar modelado, no solo sintaxis.
